<a href="https://colab.research.google.com/github/tristantoupin/ECSE415-FinalProject/blob/master/SVM_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classifer

In [13]:
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split, GridSearchCV
import cv2
import matplotlib.pyplot as plt
import os
import math
import time

### Determine whether to use the dataset from Google Cloud storage or local storage

In [2]:
# Change this value depending whether you're using a cloud or local environment
is_using_cloud = False

if is_using_cloud:
    import tarfile
    from google.colab import drive
    drive.mount('/content/gdrive')
    
    path_classification = '/content/gdrive/My Drive/Colab Notebooks/Project/MIO-TCD-Classification.tar'
    tar = tarfile.open(path_classification)
else:
    path_classification = './MIO-TCD-Classification'

### Helper functions for image plotting, importing, and preprocessing

In [3]:
def plot_images(list_of_images, max_col = 4):
    """
    @brief Plot a list of given images with
    @param list_of_images List of images to plot
    @param max_col Number of colomns to plot your figures
    """
    n = len(list_of_images)
    if n == 1:
        plt.imshow(list_of_images[0]); plt.axis('off'); plt.show()
    else:
        # get number of columns and rows required
        r, c = 1, n
        if n > max_col:
            c = max_col
            r = int(math.ceil(n/max_col))
    
        fig = plt.figure(figsize=(17, max_col * r))
        for i, (img) in enumerate(list_of_images):
            ax = fig.add_subplot(r, c, (i+1))
            ax.set_title("Image " + str(i))
            ax.axis('off')
            ax.imshow(img, cmap=plt.cm.gray)

In [4]:
def pad_resize_images(img_list, output_size):
    """
    @brief Resize each image within a list to a given shape while also keeping its aspect ratio by the use of padding
    @param img_list The list of images to resize
    @param output_size The shape of the output images are output_size x output_size
    """
    BLACK = 0
    result = np.empty_like(img_list)
    
    for i, img in enumerate(img_list):
        
        height, width = img.shape
        ratio = float(output_size) / max([height, width])
        height_new, width_new = tuple([int(val * ratio) for val in (height, width)])
        img_resized = cv2.resize(img, (height_new, width_new))
        
        height_adjust = output_size - height_new
        width_adjust = output_size - width_new
        
        top = math.ceil(height_adjust / 2)
        bot = height_adjust - top
        left = math.ceil(width_adjust / 2)
        right = width_adjust - left
        
        result[i] = cv2.resize(
            cv2.copyMakeBorder(img_resized, top, bot, left, right, cv2.BORDER_CONSTANT, value=BLACK),
            (output_size, output_size))
    return result

In [15]:
def performance_random_clf(x_tr, y_tr, x_test, y_test, r = 1234):
    clf = DummyClassifier(strategy = 'uniform', random_state = r)
    clf.fit(x_tr, y_tr)
    preds = clf.predict(x_test)
    metric = metrics.classification_report(y_test, preds)
    return metric
    
def performance_majority_classifier(x_tr, y_tr, x_test, y_test, r = 1234):
    most_common_val = stats.mode(y_tr).mode[0]
    preds = np.full((y_test.shape), most_common_val)
    metric = metrics.classification_report(y_test, preds)
    return metric

def fine_tune_svm(x_tr, y_tr, folds=10):
  # Set the parameters by cross-validation
    tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100]}]
    models = []
    clf = GridSearchCV(svm.SVC(), tuned_parameters, cv=folds,
                       scoring = 'accuracy', n_jobs = -1, verbose = 1)
    clf.fit(x_tr, y_tr)
    models.append(clf)
    print(clf.best_params_)

    return models

def compute_svm_predictions(x_train, y_train, x_val):
    model = svm.SVC()
    model.fit(x_train, y_train)
    preds = model.predict(x_val)
    return preds

#### Import a subset of or all images for each category with their relevant label

In [6]:
path_classification_train = os.path.join(path_classification, 'train')

#
# Change this number if you want more or less images. Set it to -1 if you want all images
#
number_images_category = 10
categories = os.listdir(path_classification_train)
classification_images = []
y_tr = []

start_time = time.time()
for category in categories:
    path_category = os.path.join(path_classification_train, category)
    for i, image_name in enumerate(os.listdir(path_category)):
        
        if i == number_images_category:
            break
            
        image = cv2.imread(os.path.join(path_category, image_name), cv2.IMREAD_GRAYSCALE)
        classification_images.append(image)
        
        label = categories.index(category)
        y_tr.append(label)

classification_images = np.array(classification_images)
y_tr = np.array(y_tr)

assert len(classification_images) == len(categories) * number_images_category
assert len(classification_images) == len(y_tr)

print('Elapsed Time:', time.time() - start_time)

Elapsed Time: 0.4338068962097168


#### Import a number of test images

In [7]:
path_classification_test = os.path.join(path_classification, 'test')

# For now I'll take the same amount test of images as I have for training
number_images_test = len(classification_images)
images_test = []

for i, image_name in enumerate(os.listdir(path_classification_test)):
    if i == number_images_test:
        break
    
    image = cv2.imread(os.path.join(path_classification_test, image_name), cv2.IMREAD_GRAYSCALE)
    images_test.append(image)
    
images_test = np.array(images_test)
assert len(images_test) == number_images_test

#### Pad images to maintain one common aspect ratio

In [8]:
# largest_width = np.max([x.shape[0] for x in classification_images])
# largest_height = np.max([x.shape[1] for x in classification_images])

size = 128
padded_images_train = pad_resize_images(classification_images, size)
padded_images_test = pad_resize_images(images_test, size)

#### Flatten images to prepare for SVM

In [9]:
x_tr = np.array([x.flatten() for x in padded_images_train])
y_test = np.array([x.flatten() for x in padded_images_test])

In [11]:
compute_svm_predictions(x_tr, y_tr, y_test[:10])

C:\ProgramData\Anaconda3\envs\opencv-env\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


array([10, 10, 10, 10, 10, 10, 10, 10, 10, 10])

#### Train and fine tune SVM model

In [16]:
best_svm_model = fine_tune_svm(x_tr, y_tr, folds=10)
print(best_svm_model)

Fitting 10 folds for each of 11 candidates, totalling 110 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    5.1s
[Parallel(n_jobs=-1)]: Done 110 out of 110 | elapsed:   14.5s finished


{'C': 1, 'kernel': 'linear'}
[GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid=[{'kernel': ['rbf'], 'gamma': [0.001, 0.0001], 'C': [1, 10, 100, 1000]}, {'kernel': ['linear'], 'C': [1, 10, 100]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=1)]


In [18]:
best_model = svm.SVC(C = 1, gamma = 0.001, kernel = 'linear')
best_model.fit(x_tr, y_tr)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [ ]:
preds = best_model.predict(x_val)
print(preds)